In [1]:
from retrievals import FAQDataLoader, DataPreprocessor, BM25Retrieval, EmbeddingRetrieval

In [2]:
ckip_word_segmenter_model = 'albert-tiny'
# embedding_model = 'text-embedding-3-small'

In [3]:
faq_dataLoader = FAQDataLoader(
    question_path='/home/xunhaoz/PycharmProjects/RAGAndLLMInFinance/questions_preliminary.json',
    answer_path='../contest_dataset/contest_dataset/dataset/preliminary/ground_truths_example.json',
    source_path='../contest_dataset/contest_dataset/reference/faq/pid_map_content.json'
)
data_preprocessor = DataPreprocessor(model=ckip_word_segmenter_model)
# retrieval = EmbeddingRetrieval(model=embedding_model)
retrieval = BM25Retrieval()

In [4]:
question = faq_dataLoader.get_question()
# answer = faq_dataLoader.get_answer()
source = faq_dataLoader.get_source()

In [5]:
question['clean_ws'] = data_preprocessor.preprocess(question['query'])
source['clean_ws'] = data_preprocessor.preprocess(source['content'])

Inference:   0%|          | 0/2 [00:00<?, ?it/s]

Inference: 100%|██████████| 3/3 [00:01<00:00,  1.75it/s]


In [6]:
question = retrieval.retrieval(question, source)

In [7]:
question[['retrieve']].to_csv('../faq_albert.csv')

In [8]:
(question['retrieve'] == answer['retrieve']).mean()

NameError: name 'answer' is not defined

In [ ]:
question['true retrieve'] = answer['retrieve']
question[~(question['retrieve'] == answer['retrieve'])][['query', 'retrieve', 'true retrieve']].to_csv(f'{ckip_word_segmenter_model}-error.csv')

In [ ]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

model = 'albert-tiny'

ws_driver = CkipWordSegmenter(model=model, device=0)
pos_driver = CkipPosTagger(model=model, device=0)
ner_driver = CkipNerChunker(model=model, device=0)

In [33]:
ner_driver(["資產負債表"])
ws_driver(["資產負債表"])

Inference: 100%|██████████| 1/1 [00:00<00:00, 263.79it/s]


[['資產', '負債', '表']]

In [29]:
for w, p in zip(ws_driver(["2022年金融卡不見後應該怎麼申請重新補換發卡？"]), pos_driver(ws_driver(["2022年金融卡不見後應該怎麼申請重新補換發卡？"]))):
    for ww, pp in zip(w, p):
        if pp in ['Na', 'VF', 'VC', 'VD']:
            print(f"{ww}({pp})", end=' ')

Inference: 100%|██████████| 1/1 [00:00<00:00, 290.02it/s]

金融卡(Na) 申請(VF) 補換(VC) 發(VD) 卡(Na) 